In [3]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [4]:
class Individual:
    def __init__(self, species, sex, size):
        self.species = species
        self.size = size
        self.sex = sex
        self.mating_attempts = 0

    def __repr__(self):
        return f"Ind(sp.={self.species}, sex={self.sex}, size={self.size})"

In [237]:
num_generations = 100
num_individuals = 10000
max_mating_attempts = 3
dispersal_rate = 0.1
mean_size_A = 0.4  # Mean size for species A
mean_size_B = 0.6  # Mean size for species B
std_dev = 0.1  # Standard deviation for both species
sex_ratio = 0.3

carrying_capacity = 100
reproductive_potential = 1.5
number_of_offsprings = int(reproductive_potential/sex_ratio)

density_dependent_mortality_factor = 0.0005
viablity_mortality_factor = 0.02

mate_acceptance_scaling_factor = 1.1

In [238]:
def mutual_mate_choice(male, female):


    #size adjustments to ensure within 0 and 1
    if male.size > 1:
        male.size == 1
    if female.size > 1:
        female.size == 1 
    if male.size < 0:
        male.size == 0
    if female.size < 0:
        female.size == 0

    #probability of accepting a mate each species. A prefers small, B prefers large
    if male.species == "A":
        male_accepting_probabilty = 1-female.size

    if male.species == "B":
        male_accepting_probabilty = female.size

    if female.species == "A":
        female_accepting_probabilty = 1 - male.size

    if female.species == "B":
        female_accepting_probabilty = male.size

    # print("mating attempt", male, female)


    if np.random.rand() < mate_acceptance_scaling_factor*male_accepting_probabilty*female_accepting_probabilty:
        # print("mating success")
        return True
    else:
        return False

In [239]:
def unidirectional_mate_choice(male, female):

    #size adjustments to ensure within 0 and 1
    if male.size > 1:
        male.size == 1
    if female.size > 1:
        female.size == 1 
    if male.size < 0:
        male.size == 0
    if female.size < 0:
        female.size == 0

    #probability of accepting a mate each species. A prefers small, B prefers large
    if male.species == "A":
        male_accepting_probabilty = 0.5

    if male.species == "B":
        male_accepting_probabilty = 0.5

    if female.species == "A":
        female_accepting_probabilty = 1 - male.size

    if female.species == "B":
        female_accepting_probabilty = male.size

    # print("mating attempt", male, female)


    if np.random.rand() < mate_acceptance_scaling_factor*male_accepting_probabilty*female_accepting_probabilty:
        # print("mating success")
        return True
    else:
        return False

In [240]:
#reproduction within one generation function

def reproduction_one_generation_one_patch(patch_population_list, mate_choice_function):

    np.random.seed(1)

    females_patch_list = [individual for individual in patch_population_list if individual.sex == 'female']
    males_patch_list = [individual for individual in patch_population_list if individual.sex == "male"]

    #patch A
    mating_pool_females_patch= females_patch_list.copy()
    mating_pool_males_patch = males_patch_list.copy()

    # print("mating pools male, female" , mating_pool_females_patch, mating_pool_males_patch)
  
    offspring_list = []

    while len(mating_pool_females_patch) > int(0.1*len(females_patch_list)) and len(mating_pool_males_patch) > int(0.1*len(males_patch_list)):
          #stopping rule - simulations proceed until 90% of the females have mated (might need a change)
        
        female_for_mating = np.random.choice(mating_pool_females_patch)
        male_for_mating = np.random.choice(mating_pool_males_patch)

        male_for_mating.mating_attempts += 1

        if male_for_mating.mating_attempts == max_mating_attempts:
            mating_pool_males_patch.remove(male_for_mating)
        

        if mate_choice_function(male_for_mating, female_for_mating) == True:
            mating_pool_females_patch.remove(female_for_mating)

            if male_for_mating.species == "A" and female_for_mating.species == "A":

                minimum_size_A = min(male_for_mating.size, female_for_mating.size)
                maximum_size_A = max(male_for_mating.size, female_for_mating.size)

                for k in range(number_of_offsprings):
                    random_val = np.random.beta(2, 2, 1)[0]
                    offspring = Individual(species='A', sex='female' if np.random.rand() < sex_ratio else 'male', size = random_val*minimum_size_A + maximum_size_A*(1-random_val))
                    offspring_list.append(offspring)

            if male_for_mating.species == "B" and female_for_mating.species == "B":

                minimum_size_B = min(male_for_mating.size, female_for_mating.size)
                maximum_size_B = max(male_for_mating.size, female_for_mating.size)

                for k in range(number_of_offsprings):
                    random_val = np.random.beta(2, 2, 1)[0]
                    offspring = Individual(species='B', sex='female' if np.random.rand() < sex_ratio else 'male', size = random_val*minimum_size_B + maximum_size_B*(1-random_val))
                    offspring_list.append(offspring)


    offspring_list_before_regulation = offspring_list.copy()

    # print(offspring_list_before_regulation)

    #viability selection
    random.shuffle(offspring_list_before_regulation)
    population_size_before_regulation = len(offspring_list_before_regulation)

    population_viablity_mortality_number = int(viablity_mortality_factor*population_size_before_regulation)

    if population_viablity_mortality_number > 0:
        poulation_list_after_viability_selection = offspring_list_before_regulation[:-population_viablity_mortality_number]
    else:
        poulation_list_after_viability_selection = offspring_list_before_regulation

    # print(poulation_list_after_viability_selection, "after viability")

    #density selection 
    population_size_after_viability_selection = len(poulation_list_after_viability_selection)
    population_density_mortality_number = int(density_dependent_mortality_factor*(population_size_after_viability_selection**2))

    if population_density_mortality_number > 0:
        population_list_after_density_selection = poulation_list_after_viability_selection[:-population_density_mortality_number]
    else:
        population_list_after_density_selection = poulation_list_after_viability_selection

    return population_list_after_density_selection

In [241]:
def migration_between_two_patches(list_A, list_B, dispersal_rate):

    number_of_individuals_to_dispers_from_A = int(dispersal_rate*len(list_A))

    individuals_to_disperse_from_A_to_B = list_A[:number_of_individuals_to_dispers_from_A]
    individuals_remaining_in_A = list_A[number_of_individuals_to_dispers_from_A:]

    number_of_individuals_to_dispers_from_B = int(dispersal_rate*len(list_B))

    individuals_to_disperse_from_B_to_A = list_B[:number_of_individuals_to_dispers_from_B]
    individuals_remaining_in_B = list_B[number_of_individuals_to_dispers_from_B:]
    
    list_A_new = individuals_remaining_in_A + individuals_to_disperse_from_B_to_A
    list_B_new = individuals_remaining_in_B + individuals_to_disperse_from_A_to_B
    
    return list_A_new, list_B_new

In [242]:
def reproduction_and_migration_function(patch_population_list_A, patch_population_list_B, dispersal_rate, mate_choice_function):

    patch_A_after_reproduction = reproduction_one_generation_one_patch(patch_population_list_A, mate_choice_function)
    patch_B_after_reproduction = reproduction_one_generation_one_patch(patch_population_list_B, mate_choice_function)

    patch_A_after_migration, patch_B_after_migration = migration_between_two_patches(patch_A_after_reproduction, patch_B_after_reproduction, dispersal_rate)

    return patch_A_after_migration, patch_B_after_migration

In [243]:
def count_species(individuals_list):
    count_a = sum(1 for individual in individuals_list if individual.species == "A")
    count_b = sum(1 for individual in individuals_list if individual.species == "B")
    return count_a, count_b


# Counting species A and species B


In [244]:
def reproduction_over_generations(initial_num_individuals, dispersal_rate, num_generations, mate_choice_function):

    Initial_patch_A = [Individual(species='A', sex='female' if np.random.rand() < sex_ratio else 'male', size=np.random.normal(mean_size_A, std_dev)) for _ in range(initial_num_individuals)]
    Initial_patch_B = [Individual(species='B', sex='female' if np.random.rand() < sex_ratio else 'male', size=np.random.normal(mean_size_B, std_dev)) for _ in range(initial_num_individuals)]

    patch_population_list_A = Initial_patch_A
    patch_population_list_B = Initial_patch_B

    generation_count_till_end = 0
    for i in range(num_generations):
        patch_population_list_A, patch_population_list_B = reproduction_and_migration_function(patch_population_list_A, patch_population_list_B, dispersal_rate, mate_choice_function)
        count_species_A_patch_A, count_species_B_patch_A = count_species(patch_population_list_A)
        count_species_A_patch_B, count_species_B_patch_B = count_species(patch_population_list_B)

        total_A_count = count_species_A_patch_A + count_species_A_patch_B
        total_B_count = count_species_B_patch_A + count_species_B_patch_B

        # print("PATCH A:", count_species_A_patch_A, " B count:", count_species_B_patch_A)
        # print("PATCH B:", count_species_A_patch_B, " B count:", count_species_B_patch_B)

        generation_count_till_end += 1
        
        if total_A_count == 0 or total_B_count == 0:
            break
    

    return generation_count_till_end
    print("TOTAL:" "A --", total_A_count, " B --:", total_B_count)



In [245]:
def average_number_of_generations_until_end(initial_num_individuals, dispersal_rate, num_generations, mate_choice_function):
    number_of_generations_list = []
    for i in range(5):
        number_of_generations = reproduction_over_generations(initial_num_individuals, dispersal_rate, num_generations, mate_choice_function)
        number_of_generations_list.append(number_of_generations)
    return np.mean(number_of_generations_list)

In [247]:
average_generations_until_end_of_simulation_mutual = average_number_of_generations_until_end(500, 0.15, 100, mutual_mate_choice)
print("Average number of generations until end of simulation for mutual mate choice:", average_generations_until_end_of_simulation_mutual)

average_generations_until_end_of_simulation_unidirectional = average_number_of_generations_until_end(500, 0.15, 100, unidirectional_mate_choice)
print("Average number of generations until end of simulation for unidirectional mate choice:", average_generations_until_end_of_simulation_unidirectional)

Average number of generations until end of simulation for mutual mate choice: 16.2
Average number of generations until end of simulation for unidirectional mate choice: 18.6


In [228]:
reproduction_over_generations(500, 0.15, 20, unidirectional_mate_choice)

10